### GEOGRAPHIC DATA SCIENCE - part 2 - OSM and spatial networks

Guest lecture by Sándor Juhász | [sandorjuhasz.com](sandorjuhasz.com)
<br>


In [ ]:
# if you run on Google Colab
!git clone https://github.com/sandorjuhasz/geoDS_guest_lectures.git
%cd geoDS_guest_lectures/code

!pip install osmnx

In [ ]:
# import libraries
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString, Point
import osmnx as ox
import networkx as nx 
import matplotlib.pyplot as plt

**Building footprints**

OSM is a great source of data on the built environment. In the following, we will demonstrate how to query, handle and visualize OSM data through a few examples.

In [ ]:
# requrest shape from OSM
district = ox.geocode_to_gdf("9th District, Budapest")
district.plot()

In [ ]:
# lets get the building footprint
district_polygon = district.geometry.to_list()[0]
footprint = ox.features_from_polygon(district_polygon, tags = {"building" : True})
len(footprint)

In [ ]:
footprint.plot()

In [ ]:
# lets look at what other features we got with the footprint
footprint.head(2)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(10,12))
crs = 23700

# background layer
footprint.to_crs(crs).plot(
    ax=ax,
    color="darkgrey",
    alpha=0.6,
    linewidth=0.5
)

# main layer with amenities
amenity_plot = footprint.to_crs(crs).plot(
    ax=ax,
    column="amenity",
    cmap="tab10",
    linewidth=0.5,
    edgecolor="black",
    legend=True
)

# legend
legend = amenity_plot.get_figure().axes[-1]  # access the colorbar/legend axis
ax.set_axis_off()

**Points of interest**

OSM contain a large number of destinations, amenities and other types of 'points of interests' (POIs).

They are - mostly - mapped as point geometries. To get a specific type of POI in e.g. IX. district of Budapest, we can again use `ox.geometries_from_place()` combined with a **tag dictionary**.

Lets download all shops in IX. district, Budapest which have been tagged as 'bakery'.

In [ ]:
# define tags used to query OSM
tags = {"shop": "bakery"}  

# download objects that matches query within specific location
bakeries = ox.geometries_from_place(
    ["9th District, Budapest"], tags
)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(5,5))

# plot the bakeries
bakeries.to_crs(crs).plot(ax=ax, color="orange", zorder=3)

# add the boundary
district.to_crs(crs).boundary.plot(ax=ax, color="darkgrey", zorder=2)

ax.set_axis_off()

**Street networks**

Street networks is one of the more common types of **spatial networks**. There are several different ways of defining which area we want network data for.

For example:

* Use `ox.graph_from_point()` to get the network centered around a specific location, defined by a coordinate pair.

[epsg.io](https://epsg.io/) is a great resource for finding coordinates in different coordinate reference systems (CRS).

OSMnx uses WGS84/EPSG:4326

* Use the parameters `dist` to define the search window around our point

* Set `dist_type = network` to use the distance in network length (compared to a straight "as the crow flies" distance)

* Set `network_type = walk` to only include the part of the street network where walking is allowed

In [ ]:
# coordinates of the C building of Corvinus University of Budapest
# notice that the coordinates must be passed as lat, long (y,x)
location_point = (
    47.48498593072006, # lat
    19.060465555108056, # lon
)  

# get graph within 1000 walking network distance of the C building
graph1 = ox.graph_from_point(
    location_point, dist=1000, dist_type="network", network_type="walk" 
)  

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(8, 8))

# our point coordinate location - this time lon, lat!
ax.plot(location_point[1], location_point[0], "ro", markersize=10) 

ox.plot_graph(
    graph1, ax=ax, node_color="darkblue", bgcolor="white", edge_color="darkgrey", node_size=10
)

In [ ]:
# we can turn nodes and edges to POINT and LINESTRING objects easily
nodes, edges = ox.graph_to_gdfs(graph1)
nodes.head()

In [ ]:
edges.plot(color="black", alpha=0.35)

In [ ]:
# network from address, including only nodes within 1km along the network from the address
graph_bike = ox.graph_from_address(
    address="8, Fővám tér, Budapest",
    dist=1000,
    dist_type="network",
    # this time getting all streets and paths where biking is allowed
    network_type="bike"
)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(6, 6))

# our point coordinate location 
ax.plot(location_point[1], location_point[0], "ro") 

ox.plot_graph(
    graph_bike, ax=ax, node_color="green", bgcolor="white", edge_color="darkgrey", node_size=12
);

In [ ]:
# get the Budapest metro network
# first we need to add the railway tag as a 'useful tag' to be included by OSMnx
ox.settings.useful_tags_way += [
    "railway"
]

cf = '["railway"~"subway"]'

graph_metro = ox.graph_from_place(
    ["Budapest, Hungary"],
    custom_filter=cf,
    retain_all=True,  # keep all components, not just the largest one
    truncate_by_edge=False,
    simplify=True,
)

In [ ]:
# plot the metro network of Budapest
fig, ax = plt.subplots(1,1, figsize=(8,6))
ox.plot_graph(graph_metro, ax=ax, node_size=0, node_color="darkgrey", edge_color="b", edge_linewidth=1.5)

In [ ]:
# degree table
degree_data = dict(graph_metro.degree)
degree_df = pd.DataFrame(list(degree_data.items()), columns=["osmid", "degree"]).sort_values(by="degree", ascending=False)

# add coordinates
nodes, edges = ox.graph_to_gdfs(graph_metro)
degree_df = pd.merge(
    degree_df,
    nodes,
    on="osmid",
    how="left"
)

In [ ]:
# check out the first one on google maps
degree_df.head()

In [ ]:
# final example -- Debrecen
graph_debrecen = ox.graph_from_place("Debrecen, Hungary", network_type="drive")

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(8,6))
ox.plot_graph(
    graph_debrecen, ax=ax, node_color="darkgreen", bgcolor="white", edge_color="darkgrey", node_size=2
)
ax.set_title("Debrecen")

In [ ]:
# degree distribution
degree_data = dict(graph_debrecen.degree)
degree_df = pd.DataFrame(list(degree_data.items()), columns=["osmid", "degree"]).sort_values(by="degree", ascending=False)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(5,4))
ax.hist(degree_df["degree"], color="darkgreen")
ax.set_xlabel("Degree", size=18)
ax.set_ylabel("Frequency", size=18)
ax.tick_params("y", labelsize=12)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

**Exercise**

Compare the street network of Szeged and Miskolc (or your home town)